# Setup PostgreSQL - World Cup ETL

**Auteur** : Romain  
**Date** : 16/12/2025

## Objectif
Création de la base de données PostgreSQL avec tables, partitions et index.

# 1. Installation des dépendances

In [1]:
# Installer psycopg2 si nécessaire
!pip install psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 19.1 MB/s  0:00:00 eta 0:00:01


In [28]:
import psycopg2
from psycopg2 import sql
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import pandas as pd
print(f"psycopg2 version: {psycopg2.__version__}")

psycopg2 version: 2.9.11 (dt dec pq3 ext lo64)


# 2. Configuration de connexion

In [20]:
# Créer un .env du type : 
# POSTGRES_USER=postgres
# POSTGRES_PASSWORD=password
# POSTGRES_HOST=localhost
# POSTGRES_PORT=5432
# POSTGRES_DB=dbname

from dotenv import load_dotenv
import os

# Charger les variables d'environnement depuis .env
load_dotenv()

# Récupération des variables d'environnement (avec valeurs par défaut)
DB_USER = os.getenv('DB_USER', 'postgres')
DB_PASSWORD = os.getenv('DB_PASSWORD', 'admin')
DB_HOST = os.getenv('DB_HOST', 'localhost')
DB_PORT = os.getenv('DB_PORT', '5432')
DB_NAME = os.getenv('DB_NAME', 'worldcup_db')

# Configuration de connexion
DB_CONFIG = {
    'host': DB_HOST,
    'port': DB_PORT,
    'user': DB_USER,
    'password': DB_PASSWORD
}

print(f"Configuration chargée :")
print(f"  Host: {DB_HOST}")
print(f"  Port: {DB_PORT}")
print(f"  User: {DB_USER}")
print(f"  Database: {DB_NAME}")
print(f"  Password: {'*' * len(DB_PASSWORD)}")

Configuration chargée :
  Host: localhost
  Port: 5432
  User: postgres
  Database: worldcup_db
  Password: *****


# 3. Création de la base de données

In [21]:
# Connexion au serveur PostgreSQL (sans base spécifique)
conn = psycopg2.connect(**DB_CONFIG)
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
cursor = conn.cursor()

# Vérifier si la base existe déjà
cursor.execute("SELECT 1 FROM pg_catalog.pg_database WHERE datname = %s", (DB_NAME,))
exists = cursor.fetchone()

if not exists:
    cursor.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(DB_NAME)))
    print(f"Base de données '{DB_NAME}' créée avec succès!")
else:
    print(f"La base de données '{DB_NAME}' existe déjà.")

cursor.close()
conn.close()

La base de données 'worldcup_db' existe déjà.


# 4. Fonctions utilitaires

In [22]:
def get_connection():
    """Retourne une connexion à la base de donnée"""
    return psycopg2.connect(**DB_CONFIG, database=DB_NAME)

def execute_sql(query, fetch=False):
    """Exécute une requête SQL"""
    conn = get_connection()
    cursor = conn.cursor()
    try:
        cursor.execute(query)
        conn.commit()
        if fetch:
            return cursor.fetchall()
        print("Requête exécutée avec succès")
    except Exception as e:
        conn.rollback()
        print(f"Erreur: {e}")
    finally:
        cursor.close()
        conn.close()

# Test connexion
try:
    conn = get_connection()
    print(f"Connecté à '{DB_NAME}'")
    conn.close()
except Exception as e:
    print(f"Erreur: {e}")

Connecté à 'worldcup_db'


# 5. Création de la table EQUIPE

In [24]:
sql_team = """
-- Suppression si existe
DROP TABLE IF EXISTS matches CASCADE;
DROP TABLE IF EXISTS teams CASCADE;

-- Création table teams
CREATE TABLE teams (
    id_team       SERIAL PRIMARY KEY,
    nom_standard    VARCHAR(50) NOT NULL UNIQUE,
    confederation   VARCHAR(20),
    aliases         JSONB DEFAULT '[]'::jsonb,
);

-- Index pour recherche rapide par alias (GIN pour JSONB)
CREATE INDEX idx_team_aliases ON team USING GIN (aliases);

-- Commentaires pour documentation
COMMENT ON TABLE team IS 'Référentiel des équipes nationales FIFA - World Cup 1930-2022';
COMMENT ON COLUMN team.nom_standard IS 'Nom standardisé de l''équipe (ex: Germany, France)';
COMMENT ON COLUMN team.confederation IS 'Confédération (UEFA, CONMEBOL, CAF, AFC, CONCACAF, OFC)';
COMMENT ON COLUMN team.aliases IS 'Variantes historiques du nom en JSON (ex: ["West Germany", "RFA"])';
"""

execute_sql(sql_team)
print("Table 'team' créée avec index et commentaires")

Requête exécutée avec succès
Table 'equipe' créée avec index et commentaires


# 6. Création de la table MATCH

In [25]:
sql_match = """
CREATE TABLE matches (
    id_match        SERIAL,
    home_team_id    INTEGER REFERENCES teams(id_team),
    away_team_id    INTEGER REFERENCES teams(id_team),
    home_result     INTEGER NOT NULL CHECK (home_result >= 0),
    away_result     INTEGER NOT NULL CHECK (away_result >= 0),
    result          VARCHAR(20) NOT NULL CHECK (result IN ('home_team', 'away_team', 'draw')),
    extra_time      BOOLEAN DEFAULT FALSE,
    penalties       BOOLEAN DEFAULT FALSE,
    replay          BOOLEAN DEFAULT FALSE,
    date            DATE NOT NULL,
    round           VARCHAR(50) NOT NULL,
    city            VARCHAR(100),
    id_stadium      INTEGER REFERENCES stadiums(id_stadium),
    edition         INTEGER NOT NULL,
    
    -- Éviter les doublons
    PRIMARY KEY (id_match, edition),
    CONSTRAINT unique_match UNIQUE (home_team, away_team, date, edition) --?
) PARTITION BY RANGE (edition);

-- Index pour optimisation (créés sur la table parent, propagés aux partitions)
CREATE INDEX idx_match_home_team ON match (home_team); --?
CREATE INDEX idx_match_away_team ON match (away_team); --?
CREATE INDEX idx_match_edition ON match (edition);
CREATE INDEX idx_match_date ON match (date);
CREATE INDEX idx_match_round ON match (round);
CREATE INDEX idx_match_result ON match (result);

-- Commentaires
COMMENT ON TABLE match IS 'Historique des matchs de Coupe du Monde FIFA 1930-2022';
COMMENT ON COLUMN match.result IS 'Résultat: home_team, away_team, ou draw';
COMMENT ON COLUMN match.extra_time IS 'Match décidé en prolongation';
COMMENT ON COLUMN match.penalties IS 'Match décidé aux tirs au but';
COMMENT ON COLUMN match.replay IS 'Match rejoué';
COMMENT ON COLUMN match.round IS 'Phase: Preliminary, Group Stage, Round of 16, Quarter-finals, Semi-finals, Final';
COMMENT ON COLUMN match.edition IS 'Année de l''édition';
"""

execute_sql(sql_match)
print("Table 'match' créée avec index et commentaires")

Requête exécutée avec succès
Table 'match' créée avec index et commentaires


In [26]:
# Création des partitions par édition
editions = [1930, 1934, 1938, 1950, 1954, 1958, 1962, 1966, 1970, 
            1974, 1978, 1982, 1986, 1990, 1994, 1998, 2002, 2006, 
            2010, 2014, 2018, 2022, 2026]

for i, year in enumerate(editions[:-1]):
    next_year = editions[i + 1]
    sql_partition = f"""
    CREATE TABLE IF NOT EXISTS match_{year} PARTITION OF match
        FOR VALUES FROM ({year}) TO ({next_year});
    """
    execute_sql(sql_partition)
    print(f"  Partition 'match_{year}' créée")

Requête exécutée avec succès
  Partition 'match_1930' créée
Requête exécutée avec succès
  Partition 'match_1934' créée
Requête exécutée avec succès
  Partition 'match_1938' créée
Requête exécutée avec succès
  Partition 'match_1950' créée
Requête exécutée avec succès
  Partition 'match_1954' créée
Requête exécutée avec succès
  Partition 'match_1958' créée
Requête exécutée avec succès
  Partition 'match_1962' créée
Requête exécutée avec succès
  Partition 'match_1966' créée
Requête exécutée avec succès
  Partition 'match_1970' créée
Requête exécutée avec succès
  Partition 'match_1974' créée
Requête exécutée avec succès
  Partition 'match_1978' créée
Requête exécutée avec succès
  Partition 'match_1982' créée
Requête exécutée avec succès
  Partition 'match_1986' créée
Requête exécutée avec succès
  Partition 'match_1990' créée
Requête exécutée avec succès
  Partition 'match_1994' créée
Requête exécutée avec succès
  Partition 'match_1998' créée
Requête exécutée avec succès
  Partition

# 7. Vérification de la strcuture

In [33]:
# Lister les tables
sql_tables = """
SELECT table_name 
FROM information_schema.tables 
WHERE table_schema = 'public'
ORDER BY table_name;
"""

conn = get_connection()
df_tables = pd.read_sql(sql_tables, conn)
conn.close()

print("Tables créées :")
df_tables

Tables créées :


/tmp/ipykernel_18991/2553011593.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_tables = pd.read_sql(sql_tables, conn)


,table_name
0,equipe
1,match
2,match_1930
3,match_1934
4,match_1938
5,match_1950
6,match_1954
7,match_1958
8,match_1962
9,match_1966


In [34]:
# Structure de la table equipe
sql_columns = """
SELECT column_name, data_type, is_nullable, column_default
FROM information_schema.columns
WHERE table_name = 'equipe'
ORDER BY ordinal_position;
"""

conn = get_connection()
df_equipe = pd.read_sql(sql_columns, conn)
conn.close()

print("Structure table 'equipe' :")
df_equipe

Structure table 'equipe' :


/tmp/ipykernel_18991/1270371065.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_equipe = pd.read_sql(sql_columns, conn)


,column_name,data_type,is_nullable,column_default
0,id_equipe,integer,NO,nextval('equipe_id_equipe_seq'::regclass)
1,nom_standard,character varying,NO,None
2,code_fifa,character,YES,None
3,confederation,character varying,YES,None
4,aliases,jsonb,YES,'[]'::jsonb
5,created_at,timestamp without time zone,YES,CURRENT_TIMESTAMP


In [31]:
# Structure de la table match
sql_columns = """
SELECT column_name, data_type, is_nullable, column_default
FROM information_schema.columns
WHERE table_name = 'match'
ORDER BY ordinal_position;
"""

conn = get_connection()
df_match = pd.read_sql(sql_columns, conn)
conn.close()

print("Structure table 'match' :")
df_match

Structure table 'match' :


/tmp/ipykernel_18991/4125965624.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_match = pd.read_sql(sql_columns, conn)


,column_name,data_type,is_nullable,column_default
0,id_match,integer,NO,nextval('match_id_match_seq'::regclass)
1,home_team_id,integer,YES,None
2,away_team_id,integer,YES,None
3,home_team,character varying,NO,None
4,away_team,character varying,NO,None
5,home_result,integer,NO,None
6,away_result,integer,NO,None
7,result,character varying,NO,None
8,extra_time,boolean,YES,false
9,penalties,boolean,YES,false


In [35]:
# Lister les index
sql_indexes = """
SELECT indexname, tablename
FROM pg_indexes
WHERE schemaname = 'public'
ORDER BY tablename, indexname;
"""

conn = get_connection()
df_indexes = pd.read_sql(sql_indexes, conn)
conn.close()

print(f"Index créés ({len(df_indexes)}) :")
df_indexes

Index créés (188) :


/tmp/ipykernel_18991/4056616053.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_indexes = pd.read_sql(sql_indexes, conn)


,indexname,tablename
0,equipe_nom_standard_key,equipe
1,equipe_pkey,equipe
2,idx_equipe_aliases,equipe
3,idx_equipe_code_fifa,equipe
4,idx_match_away_team,match
...,...,...
183,match_2022_home_team_away_team_date_edition_key,match_2022
184,match_2022_home_team_idx,match_2022
185,match_2022_pkey,match_2022
186,match_2022_result_idx,match_2022
